# Data Preprocessing

## Overview
In this notebook, we preprocess the Google Play and Apple Store app datasets by:
- Loading and exploring the data
- Removing incorrect data (e.g., malformed row in Google Play)
- Removing duplicates, keeping the entry with the highest number of reviews
- Filtering out non-English apps
- Isolating free apps
- Saving the preprocessed datasets to CSV files

The preprocessed data will be used for further analysis on app profitability.



## 1. Opening and Exploring the Data


Loads the raw datasets and provides an initial exploration of their structure and content.

In [ ]:
import pandas as pd

def explore_data(data_input, n_rows=5, rows_and_columns=False, header=None):
    """
    Explore a dataset using pandas.
    Prints the first n_rows in list format and optionally the number of rows and columns.

    Parameters:
    - data_input: Path to a CSV file or a pandas DataFrame
    - n_rows: Number of rows to display (default: 5)
    - rows_and_columns: If True, prints the number of rows and columns (default: False)
    - header: Column names for CSV if reading from file (default: None, uses 0-based index)

    Returns:
    - The dataframe slice or head
    """
    # Determine if input is a DataFrame or file path
    if isinstance(data_input, pd.DataFrame):
        df = data_input
    else:
        df = pd.read_csv(data_input, header=header)

    # Get the first n_rows
    df_head = df.head(n_rows)

    # Print each row as a list
    for index, row in df_head.iterrows():
        print(row.tolist())
        print()  # Adds an empty line after each row

    # Print row and column counts if requested
    if rows_and_columns:
        print('Number of rows:', len(df))
        print('Number of columns:', len(df.columns))

    # Return the head of the dataframe
    return df_head

# Load the original datasets
apple_store = pd.read_csv('/content/AppleStore.csv')
google_play = pd.read_csv('/content/googleplaystore.csv')

print("\nExploring AppleStore.csv:")
explore_data('/content/AppleStore.csv', n_rows=3, rows_and_columns=True)
print("\nExploring googleplaystore.csv:")
explore_data('/content/googleplaystore.csv', n_rows=3, rows_and_columns=True)


Exploring AppleStore.csv:
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']

Number of rows: 7198
Number of columns: 16

Exploring googleplaystore.csv:
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

['Coloring book moana', 'ART_AND_DESIGN', '

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
1,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


We can see that the AppleStore.csv dataset contains 7198 rows and 16 columns, while the googleplaystore.csv dataset has 10842 rows and 13 columns.

## 2. Deleting wrong data



The Google Play data set has a dedicated discussion section on Kaggle, and we can see that one of the discussions outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.


In [ ]:
# Print the header
print("\nHeader:")
print(google_play.columns.tolist())

# Print row 10472
print("Row 10472:")
print(google_play.iloc[10472])

# Print a correct row
print("\nRow 1 (index 0):")
print(google_play.iloc[0])



Header:
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Row 10472:
App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                               19.0
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

Row 1 (index 0):
App               Photo Edito


The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the discussions section, this problem is caused by a missing value in the 'Category' column). As a consequence, we'll delete this row.

In [ ]:
print("Original number of rows:", len(google_play))

# Delete the specific row where the app is "Life Made WI-Fi Touchscreen Photo Frame"
mask = (google_play.index == 10472) & (google_play['App'] == 'Life Made WI-Fi Touchscreen Photo Frame')
google_play = google_play[~mask]

print("New number of rows:", len(google_play))

Original number of rows: 10841
New number of rows: 10840


## 3. Removing Duplicate Entries

Some apps have more than one entry. We can calculate the total number of rows, duplicate entries, and unique duplicated apps to understand the extent of duplication in the Google Play dataset.

In [ ]:
total_rows = len(google_play)
duplicates_count = google_play[google_play.duplicated(subset=['App'], keep=False)].shape[0]
unique_duplicated_apps = len(google_play[google_play.duplicated(subset=['App'], keep=False)].groupby('App').size())
extra_duplicates = duplicates_count - unique_duplicated_apps

print(f"Total rows: {total_rows}")
print(f"Duplicate rows: {duplicates_count}")
print(f"Unique duplicated apps: {unique_duplicated_apps}")
print(f"Extra duplicate occurrences: {extra_duplicates}")
print(f"Expected unique apps: {total_rows - extra_duplicates}")

Total rows: 10840
Duplicate rows: 1979
Unique duplicated apps: 798
Extra duplicate occurrences: 1181
Expected unique apps: 9659


We can see that the total number of rows is 10840, with 1979 duplicate rows across 798 unique duplicated apps, resulting in 1181 extra duplicate occurrences. This suggests an expected unique app count of 9659 after removing duplicates.

The analysis reveals that some apps, like 'Instagram', have multiple entries with varying review counts (e.g., in the fourth column), indicating data collection at different times. The higher review count typically reflects more recent data. To address this, we prioritize keeping the entry with the highest number of reviews rather than removing duplicates randomly.

We first creates a dictionary mapping each unique app name to its highest review count.

In [ ]:
# Build the dictionary
max_reviews = google_play.groupby('App')['Reviews'].max().to_dict()

# Verify the length
print('Actual length:', len(max_reviews))

Actual length: 9659


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews

In [ ]:
import numpy as np

# Sort by 'App' and 'Reviews' in descending order to prioritize highest reviews
google_play = google_play.sort_values(by=['App', 'Reviews'], ascending=[True, False])

# Drop duplicates based on 'App', keeping the first occurrence (highest Reviews)
google_play_cleaned = google_play.drop_duplicates(subset=['App'], keep='first')

# Reset index for clean numbering
google_play_cleaned = google_play_cleaned.reset_index(drop=True)

# Print the new number of rows to verify
print("Number of rows after removing duplicates:", len(google_play_cleaned))
print("Expected number of rows (unique apps):", len(max_reviews))


Number of rows after removing duplicates: 9659
Expected number of rows (unique apps): 9659


We have 9659 rows, as expected.

## 4. Removing Non-English Apps and Isolating Free Apps

Filters out apps with more than three non-ASCII characters in their names to focus on English-language apps.

In [ ]:
# Define the is_english function to allow up to 3 non-ASCII characters
def is_english(string):
    non_ascii = sum(1 for character in str(string) if ord(character) > 127)
    return non_ascii <= 3

# Filter Android apps for English names
google_play_english = google_play_cleaned[google_play_cleaned['App'].apply(is_english)]

# Filter iOS apps for English names
apple_store_english = apple_store[apple_store['track_name'].apply(is_english)]

# Explore the filtered datasets
explore_data(google_play_english, 3, True)
print('\n')
explore_data(apple_store_english, 3, True)

['"i DT" Fútbol. Todos Somos Técnicos.', 'SPORTS', nan, 27, '3.6M', '500+', 'Free', '0', 'Everyone', 'Sports', 'October 7, 2017', '0.22', '4.1 and up', True]

['+Download 4 Instagram Twitter', 'SOCIAL', 4.5, 40467, '22M', '1,000,000+', 'Free', '0', 'Everyone', 'Social', 'August 2, 2018', '5.03', '4.1 and up', True]

['- Free Comics - Comic Apps', 'COMICS', 3.5, 115, '9.1M', '10,000+', 'Free', '0', 'Mature 17+', 'Comics', 'July 13, 2018', '5.0.12', '5.0 and up', True]

Number of rows: 9614
Number of columns: 14


[284882215, 'Facebook', 389879808, 'USD', 0.0, 2974676, 212, 3.5, 3.5, '95.0', '4+', 'Social Networking', 37, 1, 29, 1]

[389801252, 'Instagram', 113954816, 'USD', 0.0, 2161558, 1289, 4.5, 4.0, '10.23', '12+', 'Photo & Video', 37, 0, 29, 1]

[529479190, 'Clash of Clans', 116476928, 'USD', 0.0, 2130805, 579, 4.5, 4.5, '9.24.12', '9+', 'Games', 38, 5, 18, 1]

Number of rows: 6183
Number of columns: 16


,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1


We can see that we're left with 9614 Android apps and 6183 iOS apps.

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [34]:
# Filter Android apps for free ones (Price = '0')
google_play_final = google_play_english[google_play_english['Price'] == '0']

# Filter iOS apps for free ones (price = 0.0)
apple_store_final = apple_store_english[apple_store_english['price'] == 0.0]

# Print the number of free apps
print(len(apple_store_final))
print(len(google_play_final))

3222
8864


We're left with 8864 Android apps and 3222 iOS apps, which should be enough for our analysis.

## 5. Save to CSV
Exports the final preprocessed datasets to CSV files for further analysis.

In [35]:
google_play_final.to_csv('/content/drive/My Drive/preprocessed_google_play.csv', index=False)
apple_store_final.to_csv('/content/drive/My Drive/preprocessed_apple_store.csv', index=False)